In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
class MNISTData(object):
  def __init__(self):
    pass

  def load_mnist(self):
    """ load tensorflow mnist builtin dataset """
    # Split the training data to 90, and 10 %
    ds_train_s, ds_validate_s = tfds.Split.TRAIN.subsplit([9, 1])
    # Download and load three datasets directly
    tfds_train, tfds_validate, tfds_test = \
        tfds.load(name='mnist', split=[ds_train_s, ds_validate_s, tfds.Split.TEST], \
        as_supervised=True)
    return tfds_train, tfds_validate, tfds_test

In [3]:
class TfKerasModels(object):
  """ Building tf.keras models
  """
  def __init__(self):
    self.num_filters = 32
    self.kernel_size = (5, 5)
    self.pool_size = (2, 2)
    self.num_classes = 10
    self.input_shape = (28, 28, 1)

  def build_sequential_model(self):
    # Create Sequential model class:
    model = tf.keras.models.Sequential()
    # Build the model first by calling `build()` or calling `fit()` with some
    # data, or specify an `input_shape` argument in the first layer(s) for
    # automatic build.
    # Optionally,the first layer can receive an ‘input_shape` argument:
    model.add(tf.keras.layers.Conv2D(filters=self.num_filters,
      kernel_size=self.kernel_size, padding='valid', activation='relu',
      input_shape=self.input_shape))
    # Another conv2d layer
    model.add(tf.keras.layers.Conv2D(filters=self.num_filters,
      kernel_size=self.kernel_size, padding='same', activation='relu'))

    # Add a max pooling layer
    model.add(tf.keras.layers.MaxPooling2D(pool_size=self.pool_size))

    # And dropout
    model.add(tf.keras.layers.Dropout(0.5))

    # Flatten
    model.add(tf.keras.layers.Flatten())

    # Add softmax layer with 10 output units
    model.add(tf.keras.layers.Dense(self.num_classes, activation='softmax'))
    
    self._compile_model(model)
    return model

  def _compile_model(self, model):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

  def train_model(model):
    model.fit()

In [4]:
mnist_data = MNISTData()
mnist_train, mnist_validate, mnist_test = mnist_data.load_mnist()

In [5]:
BUFFER_SIZE = 10 # Use a much larger value for real code.
BATCH_SIZE = 64
NUM_EPOCHS = 5

In [6]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [7]:
train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).take(5)
validation_data = mnist_validate.map(scale).batch(BATCH_SIZE).take(5)
test_data = mnist_test.map(scale).batch(BATCH_SIZE).take(5)

STEPS_PER_EPOCH = 5

train_data = train_data.take(STEPS_PER_EPOCH)
validation_data = validation_data.take(STEPS_PER_EPOCH)
test_data = test_data.take(STEPS_PER_EPOCH)

In [8]:
keras_models = TfKerasModels()
seq_model = keras_models.build_sequential_model()

In [9]:
log_dir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
seq_model.fit(train_data, validation_data=validation_data, epochs=NUM_EPOCHS, callbacks=[tensorboard_callback])

Epoch 1/5
5/5 [==============================] - 2s 354ms/step - loss: 2.2369 - accuracy: 0.1781 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 1s 199ms/step - loss: 1.8361 - accuracy: 0.5063 - val_loss: 1.5965 - val_accuracy: 0.5344
Epoch 3/5
5/5 [==============================] - 1s 155ms/step - loss: 1.2594 - accuracy: 0.6781 - val_loss: 0.9612 - val_accuracy: 0.7375
Epoch 4/5
5/5 [==============================] - 1s 173ms/step - loss: 0.8405 - accuracy: 0.7500 - val_loss: 0.7255 - val_accuracy: 0.7344
Epoch 5/5
5/5 [==============================] - 1s 158ms/step - loss: 0.6327 - accuracy: 0.8000 - val_loss: 0.6574 - val_accuracy: 0.7906


In [11]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.
